## Script to create Foursquare dataset containing 80 Users with 200 movements each 

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.python.client import device_lib
from keras.backend import set_session
import os
import random

In [2]:
df = pd.read_csv('../datasets_produced/foursquare/dataset.csv')
df

,uid,datetime,lat,lng,vid
0,190844,2012-04-04 05:37:02,40.733596,-74.003139,3fd66200f964a52000e71ee3
1,202010,2012-04-14 23:27:41,40.733596,-74.003139,3fd66200f964a52000e71ee3
2,35452,2012-04-20 01:46:59,40.733596,-74.003139,3fd66200f964a52000e71ee3
3,121415,2012-04-21 02:46:38,40.733596,-74.003139,3fd66200f964a52000e71ee3
4,18353,2012-04-24 18:36:35,40.733596,-74.003139,3fd66200f964a52000e71ee3
...,...,...,...,...,...
33263626,124661,2013-09-16 22:29:51,40.154444,26.410847,5237865c498e89110c1d03e7
33263627,230432,2013-09-16 22:30:31,35.340099,33.309328,5237867411d2a1e910744c81
33263628,144518,2013-09-16 22:35:42,-1.404065,-48.453742,5237879111d216bab10e9e09
33263629,45417,2013-09-16 22:54:34,3.425155,-76.545010,52378c24498ea9502baf2716


In [3]:
groups = df.groupby('uid')
sizes = groups.size()
min_checkins = 200
users = sizes[sizes>=min_checkins].index 
df_min_checkins = df[df['uid'].isin(users)]
n_rows = list(range(0,200))
df_min_checkins_max_users = df_min_checkins.groupby('uid').nth(n_rows)
df_final = df_min_checkins_max_users.reset_index()
users_final = random.choices(users, k=80)
result_df = df_final[df_final.uid.isin(users_final)]
result_df['datetime'] = pd.to_datetime(result_df.datetime).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
result_df = result_df.sort_values(['uid', 'datetime'], ascending = (True, False))
vocabulary = list(result_df.vid.unique())
vid_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_vid = dict((i, c) for i, c in enumerate(vocabulary))
newList = [vid_indices[k] for k in result_df.vid if k in vid_indices]
result_df.vid = newList
result_df = result_df.astype({'vid':'int'})
result_df.to_csv(r'../datasets_final/foursquare_random_80_200/dataset.txt', sep='\t', mode='a', header=False, index=False)
result_df
# df_final

/tmp/ipykernel_2385888/4288302883.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['datetime'] = pd.to_datetime(result_df.datetime).dt.strftime('%Y-%m-%dT%H:%M:%SZ')


,uid,datetime,lat,lng,vid
8330,46,2013-09-02T11:22:58Z,-7.374963,112.793604,0
8359,46,2013-09-02T11:22:18Z,-7.374927,112.792586,1
8272,46,2013-09-02T01:29:08Z,-7.284115,112.781192,2
8215,46,2013-09-01T09:04:11Z,-7.347107,112.735268,3
8366,46,2013-08-30T08:03:06Z,-7.264267,112.740927,4
...,...,...,...,...,...
7829014,62887,2013-01-21T08:13:59Z,39.649192,27.879999,5819
7829129,62887,2013-01-15T12:41:51Z,38.438985,27.141852,2071
7829152,62887,2013-01-14T15:08:14Z,38.436848,27.142287,5790
7829126,62887,2013-01-14T08:07:41Z,38.456849,27.214980,5742


In [4]:
result_df.uid.nunique()

80